In [1]:
import brightway2 as bw
import bw2regional as bwr
import bw2_lcimpact
from copy import copy
import constructive_geometries
import os
from bw2regional.pandarus_remote import AlreadyExists

In [2]:
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 3 object(s):
	EXIOBASE 3.3.17 hybrid
	Oil seeds
	biosphere3

In [4]:
bw.methods

Methods dictionary with 0 objects

In [5]:
bwr.bw2regionalsetup('world')

In [6]:
bwr.geocollections['world']

{'filepath': '/Users/cmutel/Library/Application Support/Brightway3/EXIOBASE-regionalized-case-study.9666a926e52c1cc772cb07068bf407f2/regional/countries.gpkg',
 'field': 'isotwolettercode',
 'sha256': '654f8ee414416b3e2c2a2829cac2ea4a35c77cacb48d4ed1f58155f8dff17799',
 'kind': 'vector'}

We need to define the Rest-of-Worlds used in EXIOBASE. Happily, they are fixed, and few:

* **WA**:	RoW Asia and Pacific
* **WL**:	RoW America
* **WE**:	RoW Europe
* **WF**:	RoW Africa - really unfortunate acronym, as this is the code for an actual location...
* **WM**:	RoW Middle East

We start by gathering all the countries actually used:

We will label the EXIOBASE Rest-of-Worlds with a) a new "geocollection", and b) more suitable names.

In [7]:
exiobase_row_mapping = {
    "WA": "RoW Asia and Pacific",
    "WL": "RoW America",
    "WE": "RoW Europe",
    "WF": "RoW Africa",
    "WM": "RoW Middle East",
}

In [8]:
for activity in bw.Database("EXIOBASE 3.3.17 hybrid"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In [9]:
for activity in bw.Database("Oil seeds"):
    if activity['location'] in exiobase_row_mapping:
        activity['location'] = ('exiobase-rows', exiobase_row_mapping[activity['location']])
        activity.save()

In a separate noteboom, created GIS data for these "rest-of-world" locations.

<img src="images/exiobase-row.png">

In [10]:
bwr.geocollections['exiobase-rows'] = {
    'filepath': os.path.join(os.getcwd(), "exiobase-rows.gpkg"),
    'field': 'name'
}

In [11]:
bwr.geocollections.flush()

In [12]:
bw2_lcimpact.import_global_lcimpact()

In [18]:
bw2_lcimpact.import_regionalized_lcimpact()

/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/land_use.py:98: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/pm.py:103: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/pm.py:59: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/water.py:75: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/Users/cmutel/miniconda3/envs/regional/lib/python3.6/site-packages/bw2_lcimpact/water.py:124: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Creating intersection (world, watersheds-hh)
Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

Creating intersection (world, world-topo-particulate-matter)


/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Merging topographical faces for geocollection world


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Creating intersection (world, world-topo-ecoregions)


In [3]:
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-certain", 
    'exiobase-rows-watersheds-eq-sw-certain'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-eq-sw-all", 
    'exiobase-rows-watersheds-eq-sw-all'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "particulate-matter", 
    'exiobase-rows-particulate-matter'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "ecoregions", 
    'exiobase-rows-ecoregions'
)
bwr.remote.intersection_as_new_geocollection(
    "exiobase-rows", 
    "watersheds-hh", 
    'exiobase-rows-watersheds-hh'
)

/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Calculation job submitted.
Remote is calculating intersection, run the following when done:
                  remote.intersection(exiobase-rows-watersheds-eq-sw-all, exiobase-rows)


/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Calculation job submitted.
Remote is calculating intersection, run the following when done:
                  remote.intersection(exiobase-rows-particulate-matter, exiobase-rows)


/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Calculation job submitted.
Remote is calculating intersection, run the following when done:
                  remote.intersection(exiobase-rows-ecoregions, exiobase-rows)


/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Calculation job submitted.
Remote is calculating intersection, run the following when done:
                  remote.intersection(exiobase-rows-watersheds-hh, exiobase-rows)


# Where are crops actually grown?

Although this data source is not perfect, [Ramankutty et al](http://www.earthstat.org/harvested-area-yield-175-crops/) provide data on all our crops (note that peanuts can also be called groundnuts, as they are here).

I downloaded this data, and created compressed GeoTIFFS with `gdal_translate`:

    gdal_translate input_filename output_filename -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=DEFLATE

We also needed to transform the production rasters from total production per raster cell, to intensity (i.e. production per area, but note that this is total area, not harvested area). There is a function in `bw2regional` to do this already:

    bwr.divide_by_area(source, destination)


In [10]:
rasters = (
    ("gdpweighted.tiff", "weighted-pop-density"),
    ("oilpalm_intensity.tiff", "oilpalm"),
    ("rapeseed_intensity.tiff", "rapeseed"),
    ("soybean_intensity.tiff", "soybean"),
    ("groundnut_intensity.tiff", "peanut"),
    ("coconut_intensity.tiff", "cocnut"),
)

In [11]:
third_scales = [
    'exiobase-rows-ecoregions',
    'exiobase-rows-particulate-matter',
    'exiobase-rows-watersheds-eq-sw-all',
    'exiobase-rows-watersheds-eq-sw-certain',
    'exiobase-rows-watersheds-hh',
    'world-topo-ecoregions',
    'world-topo-particulate-matter',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-hh'
]

In [12]:
for x, y in rasters:
    bwr.geocollections[y] = {
        'filepath': os.path.abspath("data/" + x),
        'band': 1
    }

/Users/cmutel/Code/bw2/regional/bw2regional/utils.py:184: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [15]:
for fp, raster in rasters:
    for gc in third_scales:
        bwr.remote.rasterstats_as_xt(gc, raster, "xt-" + raster)